In [0]:
import pandas as pd
DATA_DIR = '/Workspace/Repos/global_nexus/databricks-hackathon-global-nexus/src/data'
RESEARCH_PAPER_DB = f'{DATA_DIR}/papers_titles_&_abstracts_1470.csv'

In [0]:
papers_df = pd.read_csv(RESEARCH_PAPER_DB)
papers_df.head()

In [0]:
import chromadb
chroma_client = chromadb.Client()
# collection = chroma_client.create_collection("ali_example_database")
collection = chroma_client.get_collection("ali_example_database")

In [0]:
# collection.add(
#     documents=[row["PAPER_TITLE"] for idx, row in papers_df.iterrows()], 
#     ids=[row["PAPER_LINK"] for idx, row in papers_df.iterrows()], 
# )

In [0]:
query= "Give me any results about methamphetamine"

In [0]:
results = collection.query(
    query_texts=[query],
    n_results=5
)

In [0]:
results

In [0]:
similarities = results['distances']
resulted_titles = results['documents']
resulted_links = results['ids']

In [0]:
rank= []
title = []
link= []
distance= []


for i in range(len(similarities[0])):
    distance.append(similarities[0][i])
    title.append(resulted_titles[0][i])
    link.append(resulted_links[0][i])
    rank.append(i+1)
 
 
#Create DataFrame
results_df = pd.DataFrame({
    'RANK': rank,
    'TITLE': title,
    'LINK': link,
    'DISTANCE': distance})


results_df

In [0]:
#Find the abstract of the resulted data
pre_results_data = results_df.merge(papers_df[['PAPER_LINK', 'PAPER_TITLE', 'PAPER_ABSTRACT']], left_on=['LINK', 'TITLE'], right_on=['PAPER_LINK', 'PAPER_TITLE'], how='left')
data_to_be_summarized = pre_results_data[['RANK', 'TITLE', 'LINK', 'DISTANCE', 'PAPER_ABSTRACT']]
data_to_be_summarized


In [0]:
data_to_be_summarized['LENGTH'] = data_to_be_summarized['PAPER_ABSTRACT'].apply(lambda x: len(x))

In [0]:
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

In [0]:
summarized_abstracts= []
for i in range(len(data_to_be_summarized)):
    abstract= data_to_be_summarized['PAPER_ABSTRACT'].iloc[i]
    summarized_abstracts.append(summarizer(abstract, max_length=130, min_length=30, do_sample=False))

In [0]:
data_to_be_summarized['PAPER_ABSTRACT'].iloc[0]

In [0]:
summarized_abstracts

In [0]:
my_series = pd.Series(summarized_abstracts)

In [0]:
my_series

In [0]:
type(my_series)

In [0]:
final_data = data_to_be_summarized [['RANK', 'TITLE', 'LINK', 'DISTANCE']]
final_data['SUMMARIZED_ABSTRACTS'] = my_series

In [0]:


final_data['SUMMARIZED_ABSTRACTS'][0][0]['summary_text']

In [0]:
final_data['SUMMARIZED_ABSTRACTS'][1][0]['summary_text']

In [0]:
for i in range(len(final_data)):
    final_data['SUMMARIZED_ABSTRACTS'].iloc[i] = final_data['SUMMARIZED_ABSTRACTS'][i][0]['summary_text']

In [0]:
final_data

CLEANED VERSION

In [0]:
# To be ran only one time

RESEARCH_PAPER_DB = '/Workspace/Repos/global_nexus/databricks-hackathon-global-nexus/src/data/papers_titles_&_abstracts_1470.csv'
papers_df = pd.read_csv(RESEARCH_PAPER_DB)
#chroma_client = chromadb.Client()
chroma_client = chromadb.PersistentClient(path="/Workspace/Repos/global_nexus/databricks-hackathon-global-nexus/src/vector_database/chroma_data_test_again/")
collection = chroma_client.create_collection("summarization_database_again")
# collection = chroma_client.get_collection("summarization_database_again")
document_list=[row["PAPER_TITLE"] for idx, row in papers_df.iterrows()] 
document_ids=[row["PAPER_LINK"] for idx, row in papers_df.iterrows()]
collection.add(
     documents=document_list,
     ids=document_ids
 )
chroma_client.list_collections()

In [0]:
import chromadb
from transformers import pipeline
import pandas as pd


RESEARCH_PAPER_DB = '/Workspace/Repos/global_nexus/databricks-hackathon-global-nexus/src/data/papers_titles_&_abstracts_1470.csv'


#def create_vector_database (data_path):

    #data = pd.read_csv(data_path)
    #chroma_client = chromadb.Client()
    #collection = chroma_client.create_collection("lllll_summarization_data_database")
    #collection.add(
    #documents=[row["PAPER_TITLE"] for idx, row in data.iterrows()], 
    #ids=[row["PAPER_LINK"] for idx, row in data.iterrows()],)

    #return collection


def load_vector_database():
    chroma_client = chromadb.PersistentClient(
        path="/Workspace/Repos/global_nexus/databricks-hackathon-global-nexus/src/vector_database/chroma_data_test_again/",
    )
    collection = chroma_client.get_collection("summarization_database_again")
    return collection


def query_vector_database (collection, query):
    #Ask Chromadb to give us a response

    results = collection.query(
    query_texts=[query],
    n_results=3)

    return results

def preprocess_query_result(results):

    #Get the data from the query results
    resulted_titles = results['documents']
    resulted_links = results['ids']

    rank= []
    title = []
    link= []


    for i in range(len(resulted_titles[0])):
        title.append(resulted_titles[0][i])
        link.append(resulted_links[0][i])
        rank.append(i+1)
 
 
    #Create DataFrame
    results_df = pd.DataFrame({
        'RANK': rank,
        'TITLE': title,
        'LINK': link
        })
    
    return results_df

def retrive_abstract_data (results_df, data_path):

    #Find the abstract of the resulted data
    data = pd.read_csv(data_path)
    pre_results_data = results_df.merge(data[['PAPER_LINK', 'PAPER_TITLE', 'PAPER_ABSTRACT']], left_on=['LINK', 'TITLE'], right_on=['PAPER_LINK', 'PAPER_TITLE'], how='left')
    data_to_be_summarized = pre_results_data[['RANK', 'TITLE', 'LINK', 'PAPER_ABSTRACT']]

    return data_to_be_summarized



def abstract_summarization(data_to_be_summarized):
    summarizer = pipeline("summarization", model="Falconsai/text_summarization") # Falconsai/text_summarization facebook/bart-large-cnn
    summarized_abstracts= []
    for i in range(len(data_to_be_summarized)):
        abstract= data_to_be_summarized['PAPER_ABSTRACT'].iloc[i]
        summarized_abstracts.append(summarizer(abstract, max_length=130, min_length=30, do_sample=False))

    my_series = pd.Series(summarized_abstracts)
    final_data = data_to_be_summarized [['RANK', 'TITLE', 'LINK']]

    final_data['SUMMARIZED_ABSTRACTS'] = my_series

    for i in range(len(final_data)):
        final_data['SUMMARIZED_ABSTRACTS'].iloc[i] = final_data['SUMMARIZED_ABSTRACTS'][i][0]['summary_text']

    return final_data

In [0]:
def summarization_pipeline (data_path):
    #data_base=create_vector_database(data_path)
    data_base = load_vector_database ()
    query_results= query_vector_database (data_base,"Give me any results about methamphetamine")
    processed_results = preprocess_query_result (query_results)
    data_to_be_summarized = retrive_abstract_data(processed_results,data_path)
    final_data= abstract_summarization(data_to_be_summarized)

    return final_data

In [0]:
summarization_results = summarization_pipeline(RESEARCH_PAPER_DB)

In [0]:
summarization_results

In [0]:
summary_list = []
for i in range(3):
    paper_row = summarization_results.iloc[i]
    paper_summary = {
        "Title": paper_row["TITLE"],
        "Link": paper_row["LINK"],
        "Summary": paper_row["SUMMARIZED_ABSTRACTS"]
    }
    summary_list.append(paper_summary)

In [0]:
summary_list

In [0]:
return_summary_string = ""
for summaries in summary_list:
    return_summary_string += summaries["Title"] + "\n"
    return_summary_string += summaries["Link"] + "\n"
    return_summary_string += summaries["Summary"] + "\n" + "\n"

In [0]:
print(return_summary_string)